## Testing a basic yield curve to zero to forward process

In [16]:
import numpy as np
import pandas as pd

In [1]:
def zero_coupon_bond_price(par, ytm, time):
    ''' Takes the par price, ytm and time to maturity and returns the spot price of the bond'''
    return par / (1 + ytm) ** time

In [11]:
par = 100
ytm = 0.01
time = 0.5
price = zero_coupon_bond_price(par, ytm, time)
price

99.50371902099891

In [5]:
price

99.6509969874776

In [13]:

market_bond_dict = [
    {
        'Face': 100,
        'Maturity': 0.25,
        'Coupon': 0,
        'Price': 97.50
    },
    {
        'Face': 100,
        'Maturity': 0.5,
        'Coupon': 0,
        'Price': 94.90
    },
    {
        'Face': 100,
        'Maturity': 1.0,
        'Coupon': 0,
        'Price': 90.00
    },
    {
        'Face': 100,
        'Maturity': 1.5,
        'Coupon': 8,
        'Price': 96.00
    },
    {
        'Face': 100,
        'Maturity': 2.0,
        'Coupon': 12,
        'Price': 101.60
    }
]

In [14]:
market_bond_dict

[{'Face': 100, 'Maturity': 0.25, 'Coupon': 0, 'Price': 97.5},
 {'Face': 100, 'Maturity': 0.5, 'Coupon': 0, 'Price': 94.9},
 {'Face': 100, 'Maturity': 1.0, 'Coupon': 0, 'Price': 90.0},
 {'Face': 100, 'Maturity': 1.5, 'Coupon': 8, 'Price': 96.0},
 {'Face': 100, 'Maturity': 2.0, 'Coupon': 12, 'Price': 101.6}]

In [18]:
market_bonds = pd.DataFrame(market_bond_dict)

In [19]:
market_bonds


,Coupon,Face,Maturity,Price
0,0,100,0.25,97.5
1,0,100,0.50,94.9
2,0,100,1.00,90.0
3,8,100,1.50,96.0
4,12,100,2.00,101.6


In [21]:
three_month_zcb_interest = 100 / 94.9
three_month_spot_rate = (4* np.log(three_month_zcb_interest))

#print('3M interest ', (three_month_zcb_interet -1)*100))

In [22]:
three_month_zcb_interest

1.053740779768177

In [23]:
three_month_spot_rate

0.20938592148883678

In [24]:
np.log(three_month_zcb_interest)

0.052346480372209195

In [25]:
def spot_rate(par, price, maturity):
    total_interest_earned = par / price
    spot = ((1/maturity) * np.log(total_interest_earned))
    total_interest_earned = total_interest_earned - 1
    
    return spot, total_interest_earned

In [28]:
spot, int = spot_rate(100,94.9,0.5)
print('6m interest =  {0:.4f}%'.format((int)*100))
print('6m spot     =  {0:.4f}%'.format(spot*100))

spot, int = spot_rate(100,90.0,1)
print('1y interest =  {0:.4f}%'.format((int)*100))
print('1y spot     =  {0:.4f}%'.format(spot*100))

6m interest =  5.3741%
6m spot     =  10.4693%
1y interest =  11.1111%
1y spot     =  10.5361%


In [29]:
market_bonds['spot_rate'] = market_bonds.apply(lambda x:
                                              spot_rate(x['Face'],
                                                       x['Price'],
                                                       x['Maturity'])[0],
                                              axis =1)

In [30]:
market_bonds

,Coupon,Face,Maturity,Price,spot_rate
0,0,100,0.25,97.5,0.101271
1,0,100,0.50,94.9,0.104693
2,0,100,1.00,90.0,0.105361
3,8,100,1.50,96.0,0.027215
4,12,100,2.00,101.6,-0.007937


In [31]:
market_bonds['coupon_freq']= 2

In [32]:
market_bonds


,Coupon,Face,Maturity,Price,spot_rate,coupon_freq
0,0,100,0.25,97.5,0.101271,2
1,0,100,0.50,94.9,0.104693,2
2,0,100,1.00,90.0,0.105361,2
3,8,100,1.50,96.0,0.027215,2
4,12,100,2.00,101.6,-0.007937,2


In [33]:
# determine the amount for each coupon payment

def bond_intermediate_coupon_npv(coupon, frequency, periods):

    coupon_amt = coupon / frequency

    coupon_npv = 0

    for i in range(1,periods):
        # range is exclusive of the end so this will give us the first and second
        # coupons which is what we need to discount; the final coupon will be
        # evaluated with the principal payment to determine the spot rate for that
        # period.

        # determine which period we are evaluating
        period = i / frequency

        # get the spot rate for the period from dataframe
        period_spot = market_bonds.loc[market_bonds['Maturity'] == period
                                       , 'spot_rate'].values

        discounted_value = coupon_amt / np.exp(period_spot * period)

        coupon_npv += discounted_value[0]
        print('NPV per {0}: {1:.2f}'.format(period, discounted_value[0]))

    print('Coupon NPV: {0:.2f}'.format(coupon_npv))
    
    return coupon_npv

In [34]:
example_bond_coupons = bond_intermediate_coupon_npv(8, 2, 3)


NPV per 0.5: 3.80
NPV per 1.0: 3.60
Coupon NPV: 7.40
